# Implementing the Transformer

Reference: [Implementation_Tutorial](Transformer_Implementation_Tutorial.ipynb)

## Attention

In [1]:
from torch import nn 
import torch

In [ ]:
class Attention(nn.module):
    def __init__(self, d_model:int, att_heads, ):
        super().__init__()

SyntaxError: expected ':' (441833506.py, line 1)